In [1]:
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

import pandas as pd
import numpy as np
#Import paths and patients classes
from notebooks.info import path_label, patient
import notebooks.utils as utils

The ROI uncertainty is to quantify how much the feature value changes between the same and different radiologists.<br>
In our case:
- For each patient, there "6" radiologists. We compute the STD among them.
- Then, we compute the mean STD as the ROI uncertainty.

In [2]:
# get features
features = pd.read_csv(repo_path / 'data' / 'features' / f'features_all_timeG.csv', index_col=0)

In [4]:
all_patients_STD = None
all_patients_mean = None

# get all values from pat_num column
pat_nums = features['pat_num'].unique()
for num in pat_nums:
    # filter only the values for the first patient
    pat_features = features[features['pat_num'] == num]
    # get STD among columns, excluding pat_num and label and keep as dataframe
    pat_features_std = pat_features.drop(['pat_num'], axis=1).std().to_frame().T
    # get mean among columns, excluding pat_num and label and keep as dataframe
    pat_features_mean = pat_features.drop(['pat_num'], axis=1).mean().to_frame().T
    # concat with all_patients_STD and all_patients_mean
    all_patients_STD = pd.concat([all_patients_STD, pat_features_std], axis=0)
    all_patients_mean = pd.concat([all_patients_mean, pat_features_mean], axis=0)
all_patients_cv = all_patients_STD / all_patients_mean
# compute mean for each feature for cv
all_patients_cv_mean = all_patients_cv.mean()

In [7]:
all_patients_STD

,shape2D_Elongation,shape2D_MajorAxisLength,shape2D_MaximumDiameter,shape2D_MeshSurface,shape2D_MinorAxisLength,shape2D_Perimeter,shape2D_PerimeterSurfaceRatio,shape2D_PixelSurface,shape2D_Sphericity,firstorder_10Percentile,...,gldm_GrayLevelNonUniformity,gldm_GrayLevelVariance,gldm_HighGrayLevelEmphasis,gldm_LargeDependenceEmphasis,gldm_LargeDependenceHighGrayLevelEmphasis,gldm_LargeDependenceLowGrayLevelEmphasis,gldm_LowGrayLevelEmphasis,gldm_SmallDependenceEmphasis,gldm_SmallDependenceHighGrayLevelEmphasis,gldm_SmallDependenceLowGrayLevelEmphasis
0,0.140270,52.745038,34.445356,5775.395245,20.649533,178.265687,0.007231,5775.221820,0.093284,6.882345,...,215.464157,6.447763,67.965420,0.487487,825.513568,0.000364,0.000138,0.026941,32.368048,0.000128
0,0.103635,24.413193,38.688669,7406.814509,26.002321,218.716754,0.009976,7406.819281,0.109386,28.600699,...,179.701082,7.480129,85.425931,0.962029,1681.106721,0.001012,0.000131,0.035735,30.792221,0.000124
0,0.106589,46.902527,49.686328,9487.289116,30.499247,196.908953,0.006396,9487.380812,0.072028,1.378405,...,377.257999,3.039363,16.533006,0.147536,173.655223,0.000159,0.000102,0.009932,12.038385,0.000102
0,0.088214,70.349114,74.016597,14603.694712,36.315009,272.878200,0.012016,14603.625925,0.110144,11.148991,...,601.154225,6.663404,213.123773,1.097898,2705.174292,0.000870,0.000295,0.046454,45.459008,0.000201
0,0.156503,70.501102,62.684324,5820.599182,28.394730,287.720279,0.017001,5820.461588,0.082276,1.826198,...,305.390700,8.751448,207.293624,0.641859,667.069326,0.002414,0.000331,0.034625,129.816230,0.000171
0,0.114173,68.200690,96.384804,28251.327478,67.689776,619.759454,0.002001,28251.257096,0.051527,97.733652,...,767.287931,17.543783,221.173974,4.609297,9913.684598,0.004668,0.000261,0.050746,30.546146,0.000142
0,0.122496,269.352692,288.814737,102878.755778,129.430841,1122.537429,0.012176,102878.834675,0.051870,12.639884,...,4714.044591,8.961930,97.452212,0.636809,287.894814,0.001563,0.000144,0.027800,63.743110,0.000118
0,0.184256,120.388238,142.244770,22127.283904,29.513583,403.712373,0.003285,22127.546612,0.087790,16.609234,...,709.134090,4.851546,107.521980,0.352228,952.354188,0.001113,0.000185,0.013986,42.221757,0.000098
0,0.144115,68.442676,84.914099,45830.158676,74.723122,328.903793,0.006453,45830.158676,0.071084,11.093542,...,2215.204257,9.113000,120.616687,1.210496,2266.740108,0.000637,0.000290,0.034526,13.083846,0.000184
0,0.144739,91.589461,81.935839,20593.118563,21.185351,245.879385,0.003274,20593.118563,0.040260,5.344779,...,922.771108,4.027791,58.681343,0.311825,682.579288,0.000260,0.000098,0.014320,15.435942,0.000070


In [10]:
# get mean among columns
all_patients_STD = all_patients_STD.mean(axis=0).to_frame().T
all_patients_STD

,shape2D_Elongation,shape2D_MajorAxisLength,shape2D_MaximumDiameter,shape2D_MeshSurface,shape2D_MinorAxisLength,shape2D_Perimeter,shape2D_PerimeterSurfaceRatio,shape2D_PixelSurface,shape2D_Sphericity,firstorder_10Percentile,...,gldm_GrayLevelNonUniformity,gldm_GrayLevelVariance,gldm_HighGrayLevelEmphasis,gldm_LargeDependenceEmphasis,gldm_LargeDependenceHighGrayLevelEmphasis,gldm_LargeDependenceLowGrayLevelEmphasis,gldm_LowGrayLevelEmphasis,gldm_SmallDependenceEmphasis,gldm_SmallDependenceHighGrayLevelEmphasis,gldm_SmallDependenceLowGrayLevelEmphasis
0,0.126836,91.985752,92.519571,22809.603144,41.274549,355.807607,0.006734,22809.580058,0.078496,11.602786,...,973.350371,5.680536,94.189796,0.706647,1274.905448,0.000957,0.000174,0.023958,33.965789,0.000121
